In [ ]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
from PIL import Image
import torch

# Load the processor and model
processor = TrOCRProcessor.from_pretrained("facebook/nougat-base")
model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-base")

# Load your image
img = Image.open("path_to_your_image.png").convert("RGB")

# Preprocess the image and prepare it for the model
pixel_values = processor(images=img, return_tensors="pt").pixel_values

# Generate predictions
generated_ids = model.generate(pixel_values)
text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(text)